In [22]:
import pandas as pd
from tqdm import tqdm
import glob
import json
import sys
sys.path.insert(0, "../bin/utils/")
import flatten as flt

In [23]:
files_paths = glob.glob("../data/raw/2016-2018_races/historic/*.json")
print(f"Extraction des données à partir de {len(files_paths)} fichiers JSON.")

Extraction des données à partir de 33558 fichiers JSON.


## Performance data extraction
Faster to work with homemade dictionnary flattening function than to use `json_normalize()`.

In [24]:
pbar = tqdm(total=len(files_paths))
performances = []
for file_path in files_paths:
    pbar.update()
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
        for horse in data["partants"]:
            horse_performance = flt.flatten_dic(horse)
            horse_performance["date"] = data["raceScheduledStartEpochMs"]
            try:
                horse_performance["priceFirst"] = data["price"]["first"]
            except:
                pass
            
            #Des fichiers n'ont pas le genyId comme raceId
            try:
                horse_performance["raceId"] = data["genyId"]
            except:
                pass
            
            performances += [horse_performance]
pbar.refresh()
df = pd.DataFrame(performances)

67116it [34:04, 32.82it/s]  


In [27]:
selected_columns = ["raceId", "date", "horse.genyId", "musique", "results.position", "priceFirst"]
df = df[selected_columns]
df.tail()

,raceId,date,horse.genyId,musique,results.position,priceFirst
403530,950616,1514734200000,2430846.0,3a0a5a0a7a9a4a4a0a6a,2.0,7650.0
403531,950616,1514734200000,2426161.0,6a1a2aDa0a6a8a5a8a9a,6.0,7650.0
403532,950616,1514734200000,2410945.0,8a5a2a5a9aDa2a5aDa3a,7.0,7650.0
403533,950616,1514734200000,2436361.0,3a2aDa2a1a1aDaDaDa4a,3.0,7650.0
403534,950616,1514734200000,2428745.0,3a3a6a1aDa3a0a3aDa9a,NaN,7650.0


In [28]:
df.to_csv("../data/interim/performances.csv")